In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import quantstats as qs
import warnings


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

closing_price = pd.read_excel('FECHAMENTO2_aula.xlsx')
quality = pd.read_excel('base_quality_limpa.xlsx')
low_size = pd.read_excel('base_low_size_limpa.xlsx')
low_vol = pd.read_excel('base_low_vol_limpa.xlsx')
momentum = pd.read_excel('base_momentum_limpa.xlsx')
value = pd.read_excel('base_value_limpa.xlsx')

In [3]:
closing_price['Data'] = pd.to_datetime(closing_price['Data'])
closing_price = closing_price.set_index('Data')

In [4]:
dados_ineg = pd.read_excel('ineg_limpo.xlsx')
dados_ineg['Data'] = pd.to_datetime(dados_ineg['Data'])
dados_ineg = dados_ineg.set_index('Data')

In [ ]:
initial_date = pd.Timestamp(dt.datetime(2017,2,2))
final_date = closing_price.index[-1]

# Colocar os lookcbacks de cada estrategia

lookback_momentum = 1  #  1 mes
lookback_low_size = 1  #  1 mes
lookback_low_vol = 1  #  1 mes
lookback_quality = 1  #  1 mes
lookback_value = 1  #  1 mes
lookback_ibx = 1  #  3 mes

rebal_time = 1  #  1 mes --> unico para toda a carteira

In [6]:
dados_ineg = dados_ineg.dropna(axis=1, how='all')

cutoff_date = initial_date
dados_ineg = dados_ineg[dados_ineg.index >= cutoff_date]

In [9]:
returns = pd.DataFrame()

while True:

    rebal = initial_date + pd.DateOffset(months=rebal_time)
    
    if rebal <= final_date:
         
        ineg_date_analysis = initial_date - pd.DateOffset(months=lookback_ibx)
        ineg = dados_ineg[(dados_ineg.index < initial_date) & (dados_ineg.index >= ineg_date_analysis)]
        ineg = ineg.bfill().ffill()

        if not ineg.empty:
            ineg = ineg.iloc[-1]
        else:
            initial_date += pd.DateOffset(months=rebal_time)
            continue

        ineg = ineg.reset_index()

        #era ticket é o nome da ação e neg é o índice de negociabilidade
        ineg.columns = ['ticker','neg'] if 'neg' not in ineg.columns else ineg.columns

        #no fim essa lista iá significar que as ações investidas estão na lista de boa negociabilidade
        ibx_tickers = ineg.sort_values(by='neg', ascending=False).head(100)['ticker'].tolist()


        #momentum:
        momentum_date_analysis = initial_date - pd.DateOffset(months=lookback_momentum)
        momentum = closing_price[(closing_price.index < initial_date) & (closing_price.index > momentum_date_analysis)]

        momentum = momentum.pct_change().add(1).cumprod().add(-1)
        momentum = momentum.iloc[-1]
        momentum = momentum.reset_index()

        #era ticket é o nome da ação e neg é o índice de negociabilidade
        momentum.columns = ['ticker','momentum']
        momentum = momentum[momentum['ticker'].isin(ibx_tickers)]

        momentum = momentum.sort_values(by='momentum', ascending=False).reset_index(drop=True)

        book_winners = momentum.ticker[:int(len(momentum)*0.4)]
        book_loosers = [l for l in list(momentum['ticker']) if l not in list(book_winners)]
        # ou 
        # book_losers = momentum.ticker[int(len(momentum))*0.4:]


        # backtest é o mesmo para winners, losers e universe
        # universe
        backtest_universe = closing_price[(closing_price.index >= initial_date) & (closing_price.index < rebal)]
        backtest_universe = backtest_universe.pct_change().replace(np.nan,0)
        backtest_universe['Retorno'] = backtest_universe.mean(axis=1)
        backtest_universe_ret = backtest_universe['Retorno'][1:]

        # losers
        backtest_loosers = closing_price[(closing_price.index >= initial_date) & (closing_price.index < rebal)]
        backtest_loosers = backtest_loosers[book_loosers].pct_change().replace(np.nan,0)
        backtest_loosers['Retorno'] = backtest_loosers.mean(axis=1)
        backtest_loosers_ret = backtest_loosers['Retorno'][1:]

        # winners
        backtest_winners = closing_price[(closing_price.index >= initial_date) & (closing_price.index < rebal)]
        backtest_winners = backtest_winners[book_winners].pct_change().replace(np.nan,0)
        backtest_winners['Retorno'] = backtest_winners.mean(axis=1)
        backtest_winners_ret = backtest_winners['Retorno'][1:]

        # retorno
        retorno_rebal = pd.DataFrame({
            'Winners': backtest_winners_ret,
            'Universo': backtest_universe_ret,
            'Losers': backtest_loosers_ret,
        })


        returns = pd.concat([returns, retorno_rebal], ignore_index=False)

        initial_date = initial_date + pd.DateOffset(months=rebal_time)

    else:
        break

returns = returns.add(1).cumprod()

qs.reports.full(returns['Winners'],returns['Universo'])


AttributeError: module 'quantstats' has no attribute 'reports'